In [ ]:
#!pip install pandas numpy matplotlib statsbombpy

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsbombpy import sb

In [ ]:
from statsbombpy import sb

# 1. Retrieve competition + season IDs for World Cup 2022
comps = sb.competitions()
wc = comps[(comps.competition_name == 'FIFA World Cup') & (comps.season_name == '2022')]
print(wc[['competition_id','season_id']])
# Expected: competition_id = 43, season_id = 106

# 2. List all World Cup 2022 matches
matches = sb.matches(competition_id=43, season_id=106)

# 3. Filter for the final between Argentina (home) and France (away)
final = matches[(matches.home_team == 'Argentina') & (matches.away_team == 'France')]
print(final[['match_id','match_date','competition_stage']])


Prepare the data

In [ ]:
#load event data
events = sb.events(match_id = 3869685)

# Extract Pass Events
passes = events[events['type'] == 'Pass']

# Clean pass data 
passes_df = passes[['team', 'player', 'location', 'pass_end_location', 'minute']].copy()

# Drop rows with missing or invalid locations
mask = (
    passes_df['location'].apply(lambda x: isinstance(x, list) and len(x) == 2) &
    passes_df['pass_end_location'].apply(lambda x: isinstance(x, list) and len(x) == 2)
)
passes_df = passes_df[mask].reset_index(drop=True)

# Split location columns
loc_df = pd.DataFrame(passes_df['location'].tolist(), columns=['x_start', 'y_start'])
end_loc_df = pd.DataFrame(passes_df['pass_end_location'].tolist(), columns=['x_end', 'y_end'])
passes_df = pd.concat([passes_df, loc_df, end_loc_df], axis=1)
passes_df.drop(['location', 'pass_end_location'], axis=1, inplace=True)

# Save to CSV
passes_df.to_csv('../data/passes.csv', index=False)

# Extract Ball Touches (for heatmaps)
touches = events[['team', 'player', 'location', 'minute']]
touches = touches[touches['location'].apply(lambda x: isinstance(x, list) and len(x) == 2)]
touches[['x', 'y']] = pd.DataFrame(touches['location'].tolist(), index=touches.index)
touches.drop(['location'], axis=1, inplace=True)
touches.to_csv('../data/touches.csv', index=False)
